In [2]:
import ee

In [3]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWipNofJgsNrKkyxuIbGfz65hnLj10L50S-9C93kmSYEYA4J68FFz24

Successfully saved authorization token.


In [4]:
# Print ketingian gunung gede.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([106.9775661, -6.7881845])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Ketinggian Gunung Gede  (m):', elev)

Ketinggian Gunung Gede  (m): 2746


In [14]:
import folium

# Bogor
lat, lon = -6.5951935,106.791892

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [16]:
import ee 
import folium 

# Inisialisasi library. 
ee.Initialize() 
# TRMM 3B43: Perkiraan Presipitasi Bulanan (PPB)
# Dari dokumentasi https://developers.google.com/earth-engine/datasets/catalog/TRMM_3B43V7
# dataset ini memiliki 3 band atau atribut yaitu precipitation, relativeError dan gaugeRelativeWeighting
PPB = ee.ImageCollection("TRMM/3B43V7") 
# Tentukan waktu dan daerah kajian 
tanggal_awal = '1998-01-01' 
tanggal_akhir = '2019-12-01' 
Jawa = ee.FeatureCollection('users/heryhebho/Jawa_ajha') 
# Dari dataset Presipitasi_Bulanan, pilih atribut "precipitation", 
# batasi data pada rentang tanggal_awal sampai tanggal_akhir 
# kemudian hitung rataan-nya 21 
Rataan_PPB = PPB.select('precipitation').filterDate(tanggal_awal, tanggal_akhir).mean() 


# Potong daerah kajian 
Rataan_PPB_Jawa = Rataan_PPB.clip(Jawa) 

# Atur parameter visualisasi 
Rataan_PPB_visParam = { 
    'min': 0,'max': 0.7, 'palette': ['yellow', 'blue'], 'opacity':0.5
}

# Buat method untuk memasukkan layar
def add_ee_layer(self, ee_image_object, vis_params, name): 
    """Adds a method for displaying Earth Engine image tiles to folium map.""" 
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', 
        name=name, 
        overlay=True, 
        control=True
    ).add_to(self)
    
# Tambahankan method add_ee_layer ke dalam folium
folium.Map.add_ee_layer = add_ee_layer

# Membuat peta Jawa dengan koordinat dari Google Map (Java/@-7.3226256,107.6630558,7z) 
lat, lon = -7.32, 108 
my_map = folium.Map(location=[lat, lon], zoom_start=7)

# Tambahkan layer Rataan_PPB_Jawa. 
my_map.add_ee_layer(Rataan_PPB_Jawa, Rataan_PPB_visParam , 'Rataan Presipitasi Bulanan 53 1998-2019') 

# Tambahan Layer Control
my_map.add_child(folium.LayerControl()) 

# Tampilkan peta 
display(my_map)